# 必要なパッケージのインストール

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import pickle
import os
import pandas as pd
import time, datetime

# investing.comへのクローラーの設定

## driverの設定

In [175]:
# headless optionの設定
options = Options()

# headlessの設定
# options.add_argument("--headless");

# ファイルダウンロード先フォルダの設定
download_dir = os.path.join(os.getcwd(), "output/investing_com_historical_stock_price")
os.makedirs(download_dir, exist_ok=True) #フォルダ作成
options.add_experimental_option("prefs",{"download.default_directory" : download_dir})

# driverの定義
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [176]:
wait = WebDriverWait(driver, 10)

## investing.comへアクセス

In [23]:
url = "https://www.investing.com/"

In [24]:
# investing.comのTopページへアクセス
driver.get(url)

# ログイン

## ログイン情報の定義

In [96]:
mail_str = "masa.ochi.74@gmail.com"
pswd_str = "1031projectB"

## ログイン操作

In [97]:
login_btn = driver.find_element_by_css_selector("div.topBarText > a.login.bold")
login_btn.click()

ElementClickInterceptedException: Message: element click intercepted: Element <a onclick="overlay.overlayLogin();" href="javascript:void(0);" class="login bold">...</a> is not clickable at point (821, 51). Other element would receive the click: <div class="generalOverlay js-general-overlay displayNone" style="display: block;"></div>
  (Session info: chrome=86.0.4240.183)


In [98]:
login_form = driver.find_element_by_css_selector("form#loginPopupform")
# メールアドレスの入力
form_mail = login_form.find_element_by_css_selector("input#loginFormUser_email")
form_mail.send_keys(mail_str)

# パスワードの入力
form_pswd = login_form.find_element_by_css_selector("input#loginForm_password")
form_pswd.send_keys(pswd_str)

# エンター押下
form_pswd.send_keys(Keys.RETURN)

# 銘柄のhistoryデータ取得

## S&P 500 銘柄のURLリスト取得

In [106]:
driver.get("https://www.investing.com/equities/united-states")

In [109]:
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'selectBox')))
pull_down = Select(driver.find_element_by_css_selector(".selectBox"))
pull_down.select_by_index(4)
driver.implicitly_wait(30)

In [110]:
stock_url_lst = []
for el in driver.find_elements_by_css_selector("table.genTbl > tbody > tr > td > a"):
    stock_url_lst.append(el.get_attribute("href"))

In [3]:
len(stock_url_lst)

581

In [112]:
with open("./tmp/stock_price_history_url_lst.pkl", "wb") as f:
    pickle.dump(stock_url_lst, f)

In [2]:
with open("./tmp/stock_price_history_url_lst.pkl", "rb") as f:
    stock_url_lst = pickle.load(f)

## ヒストリーデータの取得

### ヒストリーデータ参照画面へ遷移

In [113]:
driver.get(stock_url_lst[0])

#### 銘柄Symbolの取得

In [114]:
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'newBigSubTabs')))

<selenium.webdriver.remote.webelement.WebElement (session="4f49e9348c77bcddf0d8700336c560b4", element="7a743ff4-fc7d-4eae-b3dc-131958c5b01a")>

In [115]:
symbol_name = driver.find_element_by_css_selector(".float_lang_base_1.inlineblock").text.split(" ")[0]

In [116]:
historical_data_url = driver.find_element_by_css_selector(".arial_12.newBigSubTabs > li:nth-child(3) > a").get_attribute("href")
driver.get(historical_data_url)

### 期間を指定

In [117]:
calender_btn = driver.find_element_by_css_selector("div.float_lang_base_1#widget")
calender_btn.click()

driver.implicitly_wait(10)
start_date_input = driver.find_element_by_css_selector("input.newInput#startDate")
start_date_input.clear()
start_date_input.send_keys("01/01/2018")

end_date_input = driver.find_element_by_css_selector("input.newInput#endDate")
end_date_input.clear()
end_date_input.send_keys("09/30/2020")

apply_btn = driver.find_element_by_css_selector(".newBtn.Arrow.LightGray.float_lang_base_2")
apply_btn.click()

### ファイルダウンロード

In [118]:
download_btn = driver.find_element_by_css_selector(".newBtn.LightGray.downloadBlueIcon.js-download-data")
download_btn.click()

# ファイルダウンロードのための関数作成

In [2]:
def login_investing_com():
    url = "https://www.investing.com/"
    driver.get(url)
    
    mail_str = "masa.ochi.74@gmail.com"
    pswd_str = "1031projectB"
    
    login_btn = driver.find_element_by_css_selector("div.topBarText > a.login.bold")
    login_btn.click()
    login_form = driver.find_element_by_css_selector("form#loginPopupform")
    
    # メールアドレスの入力
    form_mail = login_form.find_element_by_css_selector("input#loginFormUser_email")
    form_mail.send_keys(mail_str)

    # パスワードの入力
    form_pswd = login_form.find_element_by_css_selector("input#loginForm_password")
    form_pswd.send_keys(pswd_str)

    # エンター押下
    form_pswd.send_keys(Keys.RETURN)

In [11]:
# 詳細TOPページのURLを引数にとる関数
def get_stock_price_history(url):
    driver.get(url)
    
    driver.implicitly_wait(30)
    symbol_name = driver.find_element_by_css_selector(".float_lang_base_1.inlineblock").text.split(" ")[0]
    driver.find_element_by_css_selector(".arial_14.bold.newBigTabs > li:nth-child(1)").click()
    historical_data_url = driver.find_element_by_css_selector(".arial_12.newBigSubTabs > li:nth-child(3) > a").get_attribute("href")
    driver.get(historical_data_url)
    
    calender_btn = driver.find_element_by_css_selector("div.float_lang_base_1#widget")
    calender_btn.click()

    start_date_input = driver.find_element_by_css_selector("input.newInput#startDate")
    start_date_input.clear()
    start_date_input.send_keys("01/01/2018")

    end_date_input = driver.find_element_by_css_selector("input.newInput#endDate")
    end_date_input.clear()
    end_date_input.send_keys("09/30/2020")

    apply_btn = driver.find_element_by_css_selector(".newBtn.Arrow.LightGray.float_lang_base_2")
    apply_btn.click()
    
    time.sleep(5)
    download_btn = driver.find_element_by_css_selector(".newBtn.LightGray.downloadBlueIcon.js-download-data")
    download_btn.click()
    
    print("{0}: {1} is downloaded...!".format(datetime.datetime.now(), symbol_name))

In [21]:
# headless optionの設定
options = Options()

# headlessの設定
# options.add_argument("--headless");

# ファイルダウンロード先フォルダの設定
download_dir = os.path.join(os.getcwd(), "output/investing_com_historical_stock_price")
os.makedirs(download_dir, exist_ok=True) #フォルダ作成
options.add_experimental_option("prefs",{"download.default_directory" : download_dir})

# driverの定義
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)

login_investing_com()

get_stock_price_history(stock_url_lst[495])

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


2020-11-05 17:53:12.561152: QCOM is downloaded...!


In [7]:
# tmp_df = pd.read_table(
#     os.path.join("./output/investing_com_historical_stock_price", 
#                  os.listdir("./output/investing_com_historical_stock_price")[1]), sep=",", header=0)

In [13]:
stock_url_lst.index("https://www.investing.com/equities/qualcomm-inc-earnings")

495

In [27]:
stock_url_lst

['https://www.investing.com/equities/apple-computer-inc',
 'https://www.investing.com/equities/general-electric',
 'https://www.investing.com/equities/bank-of-america',
 'https://www.investing.com/equities/adv-micro-device',
 'https://www.investing.com/equities/ford-motor-co',
 'https://www.investing.com/equities/american-airlines-group',
 'https://www.investing.com/equities/wells-fargo',
 'https://www.investing.com/equities/facebook-inc',
 'https://www.investing.com/equities/twitter-inc',
 'https://www.investing.com/equities/at-t',
 'https://www.investing.com/equities/microsoft-corp',
 'https://www.investing.com/equities/marathon-oil',
 'https://www.investing.com/equities/exxon-mobil',
 'https://www.investing.com/equities/pfizer',
 'https://www.investing.com/equities/abbvie-inc',
 'https://www.investing.com/equities/citigroup',
 'https://www.investing.com/equities/intel-corp',
 'https://www.investing.com/equities/carnival-corp-exch',
 'https://www.investing.com/equities/bristol-myer-s

In [ ]:
# headless optionの設定
options = Options()

# headlessの設定
# options.add_argument("--headless");

# ファイルダウンロード先フォルダの設定
download_dir = os.path.join(os.getcwd(), "output/investing_com_historical_stock_price")
os.makedirs(download_dir, exist_ok=True) #フォルダ作成
options.add_experimental_option("prefs",{"download.default_directory" : download_dir})

# driverの定義
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)

login_investing_com()
i = 496
while i < len(stock_url_lst):
    try:
        get_stock_price_history(stock_url_lst[i])
        i = i + 1
    except:
        pass

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


2020-11-05 17:54:38.431488: MELI is downloaded...!
2020-11-05 17:54:52.515303: BIDU is downloaded...!
2020-11-05 17:55:06.435107: PSA is downloaded...!
2020-11-05 17:55:21.456470: MET is downloaded...!
2020-11-05 17:55:39.502782: VRSK is downloaded...!
2020-11-05 17:55:59.617828: ALL is downloaded...!
2020-11-05 17:56:18.487404: AWK is downloaded...!
2020-11-05 17:56:31.581298: ANSS is downloaded...!
2020-11-05 17:56:47.274641: HLT is downloaded...!
2020-11-05 17:57:01.064664: HOLX is downloaded...!
2020-11-05 17:57:16.898006: EXR is downloaded...!
2020-11-05 17:57:28.990533: QRVO is downloaded...!
2020-11-05 17:58:20.148578: DISH is downloaded...!
2020-11-05 17:58:38.824242: PXD is downloaded...!
2020-11-05 17:58:52.850458: EXPE is downloaded...!
2020-11-05 17:59:07.022709: BMRN is downloaded...!
2020-11-05 17:59:22.128336: JKHY is downloaded...!
2020-11-05 17:59:33.910296: LBTYA is downloaded...!
2020-11-05 17:59:47.124740: LBTYK is downloaded...!
2020-11-05 17:59:59.843362: ALB is d

In [ ]:
driver.save_screenshot('search_results.png') 

In [15]:
duplicated_file = [s for s in os.listdir("./output/investing_com_historical_stock_price") if '(1).csv' in s]

In [16]:
duplicated_file

['AWK Historical Data (1).csv',
 'JKHY Historical Data (1).csv',
 'DISH Historical Data (1).csv',
 'APA Historical Data (1).csv',
 'ANSS Historical Data (1).csv',
 'HST Historical Data (1).csv',
 'EXPE Historical Data (1).csv',
 'ALB Historical Data (1).csv',
 'EXR Historical Data (1).csv',
 'LNC Historical Data (1).csv',
 'VRSK Historical Data (1).csv',
 'LUMN Historical Data (1).csv',
 'PRGO Historical Data (1).csv',
 'MRO Historical Data (1).csv',
 'PSA Historical Data (1).csv',
 'MET Historical Data (1).csv',
 'HLT Historical Data (1).csv',
 'ALL Historical Data (1).csv',
 'CF Historical Data (1).csv',
 'QRVO Historical Data (1).csv',
 'HOLX Historical Data (1).csv',
 'XEC Historical Data (1).csv',
 'PXD Historical Data (1).csv']

In [18]:
os.remove([os.path.join("./output/investing_com_historical_stock_price", s) for s in duplicated_file])

TypeError: remove: path should be string, bytes or os.PathLike, not list

In [25]:
for i in [os.path.join("./output/investing_com_historical_stock_price", s) for s in duplicated_file]:
    os.remove(i)

# csvファイルのデータ読み込み

In [40]:
os.chdir("./output/investing_com_historical_stock_price")
csv_files = os.listdir(".")

In [51]:
csv_files[0].split(" ")[0]

'PEP'

In [53]:
tmp_df = pd.read_table(csv_files[0], sep=",", header=0)
tmp_df["Symbol"] = csv_files[0].split(" ")[0]
df = tmp_df

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [55]:
for f in csv_files[1:]:
    tmp_df = pd.read_table(f, sep=",", header=0)
    tmp_df["Symbol"] = f.split(" ")[0]
    df = df.append(tmp_df)

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [58]:
df.shape

(344311, 8)

In [107]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %,Symbol
0,"Sep 30, 2020",138.6,137.96,139.19,136.93,7.06M,1.05%,PEP
1,"Sep 29, 2020",137.16,138.72,139.54,137.15,4.33M,-0.59%,PEP
2,"Sep 28, 2020",137.97,135.96,138.58,135.75,7.49M,3.31%,PEP
3,"Sep 25, 2020",133.55,130.89,133.83,130.51,4.58M,1.50%,PEP
4,"Sep 24, 2020",131.58,131.27,132.31,129.94,4.32M,0.44%,PEP


In [59]:
df.describe()

,Date,Price,Open,High,Low,Vol.,Change %,Symbol
count,344311,344311.00,344311.0,344311.0,344311.0,344311,344311,344311
unique,708,43245.00,42559.0,43066.0,42517.0,56056,3601,500
top,"Sep 23, 2020",53.17,80.0,45.0,28.0,1.03M,0.00%,CBOE
freq,500,48.00,98.0,59.0,55.0,1112,1795,708


In [94]:
np.array((df.groupby(["Symbol", "Date"]).size() > 1).to_list()).any()

False

In [96]:
os.chdir("../../")
df_s_and_p_stock_list = pd.read_table("./output/s_and_p_stock_list.csv", sep=",", header=0)

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [99]:
s_and_p_stock_symbol = df_s_and_p_stock_list["Symbol"]

In [104]:
df.query("Symbol not in @s_and_p_stock_symbol")["Symbol"].nunique()

22

In [105]:
df.query("Symbol in @s_and_p_stock_symbol")["Symbol"].nunique()

478

# dfデータの整形

In [108]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %,Symbol
0,"Sep 30, 2020",138.6,137.96,139.19,136.93,7.06M,1.05%,PEP
1,"Sep 29, 2020",137.16,138.72,139.54,137.15,4.33M,-0.59%,PEP
2,"Sep 28, 2020",137.97,135.96,138.58,135.75,7.49M,3.31%,PEP
3,"Sep 25, 2020",133.55,130.89,133.83,130.51,4.58M,1.50%,PEP
4,"Sep 24, 2020",131.58,131.27,132.31,129.94,4.32M,0.44%,PEP


In [119]:
df = df.rename(columns={"Date": "Date_str"})

In [120]:
df.head()

,Date_str,Price,Open,High,Low,Vol.,Change %,Symbol
0,"Sep 30, 2020",138.6,137.96,139.19,136.93,7.06M,1.05%,PEP
1,"Sep 29, 2020",137.16,138.72,139.54,137.15,4.33M,-0.59%,PEP
2,"Sep 28, 2020",137.97,135.96,138.58,135.75,7.49M,3.31%,PEP
3,"Sep 25, 2020",133.55,130.89,133.83,130.51,4.58M,1.50%,PEP
4,"Sep 24, 2020",131.58,131.27,132.31,129.94,4.32M,0.44%,PEP


In [121]:
df = df.assign(Date = pd.to_datetime(df['Date_str']))

In [122]:
df.head()

,Date_str,Price,Open,High,Low,Vol.,Change %,Symbol,Date
0,"Sep 30, 2020",138.6,137.96,139.19,136.93,7.06M,1.05%,PEP,2020-09-30
1,"Sep 29, 2020",137.16,138.72,139.54,137.15,4.33M,-0.59%,PEP,2020-09-29
2,"Sep 28, 2020",137.97,135.96,138.58,135.75,7.49M,3.31%,PEP,2020-09-28
3,"Sep 25, 2020",133.55,130.89,133.83,130.51,4.58M,1.50%,PEP,2020-09-25
4,"Sep 24, 2020",131.58,131.27,132.31,129.94,4.32M,0.44%,PEP,2020-09-24


In [123]:
df.tail()

,Date_str,Price,Open,High,Low,Vol.,Change %,Symbol,Date
687,"Jan 08, 2018",308.26,308.95,309.34,307.56,660.79K,0.01%,NOC,2018-01-08
688,"Jan 05, 2018",308.22,309.09,310.24,307.21,842.80K,-0.11%,NOC,2018-01-05
689,"Jan 04, 2018",308.55,306.58,308.71,305.61,582.40K,0.62%,NOC,2018-01-04
690,"Jan 03, 2018",306.65,304.74,307.25,304.61,722.05K,0.46%,NOC,2018-01-03
691,"Jan 02, 2018",305.24,307.35,307.72,303.21,715.00K,-0.54%,NOC,2018-01-02


In [139]:
df = df.assign(Volume = df["Vol."].apply(lambda a: str(a).replace("K", "0").replace("M", "0000").replace(".", "")))

In [140]:
df.head()

,Date_str,Price,Open,High,Low,Vol.,Change %,Symbol,Date,Volume
0,"Sep 30, 2020",138.6,137.96,139.19,136.93,7.06M,1.05%,PEP,2020-09-30,7060000
1,"Sep 29, 2020",137.16,138.72,139.54,137.15,4.33M,-0.59%,PEP,2020-09-29,4330000
2,"Sep 28, 2020",137.97,135.96,138.58,135.75,7.49M,3.31%,PEP,2020-09-28,7490000
3,"Sep 25, 2020",133.55,130.89,133.83,130.51,4.58M,1.50%,PEP,2020-09-25,4580000
4,"Sep 24, 2020",131.58,131.27,132.31,129.94,4.32M,0.44%,PEP,2020-09-24,4320000


In [141]:
df.tail()

,Date_str,Price,Open,High,Low,Vol.,Change %,Symbol,Date,Volume
687,"Jan 08, 2018",308.26,308.95,309.34,307.56,660.79K,0.01%,NOC,2018-01-08,660790
688,"Jan 05, 2018",308.22,309.09,310.24,307.21,842.80K,-0.11%,NOC,2018-01-05,842800
689,"Jan 04, 2018",308.55,306.58,308.71,305.61,582.40K,0.62%,NOC,2018-01-04,582400
690,"Jan 03, 2018",306.65,304.74,307.25,304.61,722.05K,0.46%,NOC,2018-01-03,722050
691,"Jan 02, 2018",305.24,307.35,307.72,303.21,715.00K,-0.54%,NOC,2018-01-02,715000


In [142]:
df_save = df[["Symbol", "Open", "Price", "Low", "High", "Volume", "Date"]]

In [145]:
df_save.columns = ["ticker", "open", "close", "low", "high", "volume", "date"]

In [146]:
df_save.head()

,ticker,open,close,low,high,volume,date
0,PEP,137.96,138.6,136.93,139.19,7060000,2020-09-30
1,PEP,138.72,137.16,137.15,139.54,4330000,2020-09-29
2,PEP,135.96,137.97,135.75,138.58,7490000,2020-09-28
3,PEP,130.89,133.55,130.51,133.83,4580000,2020-09-25
4,PEP,131.27,131.58,129.94,132.31,4320000,2020-09-24


In [147]:
df_save.to_csv("./output/s_and_p_stock_price_history_from_investing_com.csv")